## **Install all the Dependencies**

In [1]:
# Install compatible versions
!pip install huggingface_hub
!pip install sentence-transformers
!pip install gradio sentence_transformers transformers rouge_score nltk bert_score Levenshtein POT==0.4.0 sacremoses
!pip install --upgrade python-docx

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.

## **Project Implementation**

In [2]:
from transformers import MarianMTModel, MarianTokenizer
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import Levenshtein
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from sklearn.preprocessing import MinMaxScaler
import torch
from gensim.models import KeyedVectors
import numpy as np
import gradio as gr

# Download NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Language Codes
language_codes = {
    "English": "en",
    "Hindi": "hi",
    "Spanish": "es",
    "French": "fr",
    "German": "de",
    "Italian": "it",
    "Portuguese": "pt",
    "Dutch": "nl",
    "Russian": "ru",
    "Chinese": "zh",
    "Japanese": "ja",
    "Korean": "ko",
    "Arabic": "ar",
    "Bengali": "bn",
    "Turkish": "tr",
    "Urdu": "ur",
    "Persian": "fa",
    "Swahili": "sw",
    "Tamil": "ta",
    "Telugu": "te",
    "Vietnamese": "vi",
    "Thai": "th",
    "Greek": "el",
    "Hebrew": "he",
    "Polish": "pl",
    "Romanian": "ro",
    "Hungarian": "hu",
    "Czech": "cs",
    "Swedish": "sv",
    "Danish": "da",
    "Finnish": "fi",
    "Norwegian": "no",
    "Icelandic": "is",
}

# Load Translation Model
def load_translation_model(source_lang, target_lang):
    try:
        model_name = f'Helsinki-NLP/opus-mt-{source_lang}-{target_lang}'
        model = MarianMTModel.from_pretrained(model_name)
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        return model, tokenizer
    except Exception as e:
        return None, None

# Translate Text
def translate_text(text, model, tokenizer):
    encoded_text = tokenizer([text], return_tensors='pt', padding=True)
    translation = model.generate(**encoded_text)
    return tokenizer.decode(translation[0], skip_special_tokens=True)

# BLEU Score
def calculate_bleu(reference_text, generated_text):
    reference = [reference_text.split()]
    candidate = generated_text.split()
    return sentence_bleu(reference, candidate)

# ROUGE Score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
def calculate_rouge(reference_text, generated_text):
    return scorer.score(reference_text, generated_text)

# METEOR Score
def calculate_meteor(reference_text, generated_text):
    # Tokenize the generated text
    generated_tokens = nltk.word_tokenize(generated_text)
    reference_tokens = nltk.word_tokenize(reference_text)
    return meteor_score([reference_tokens], generated_tokens)

# Levenshtein Distance
def calculate_levenshtein_distance(text1, text2):
    return Levenshtein.distance(text1, text2)

# Cosine Similarity
def calculate_cosine_similarity(text1, text2):
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)[0, 1]

# Sentence Embedding Similarity
model_sbert = SentenceTransformer('paraphrase-MiniLM-L6-v2')
def calculate_sentence_embedding_similarity(text1, text2):
    embedding1 = model_sbert.encode(text1)
    embedding2 = model_sbert.encode(text2)
    return util.pytorch_cos_sim(embedding1, embedding2).item()

# Normalize Scores
def normalize_scores(scores):
    scaler = MinMaxScaler()
    scores = [[score] for score in scores]
    return [score[0] for score in scaler.fit_transform(scores)]

# Adjust Weights
def adjust_weights_for_task(task_type='translation'):
    if task_type == 'translation':
        return [1, 1, 1, 1, 1, 1, 1]
    elif task_type == 'summarization':
        return [0.1, 0.3, 0.2, 0.1, 0.1, 0.1, 0.1]

# Aggregate Score
def aggregate_score(scores, weights):
    normalized_scores = normalize_scores(scores)
    weighted_sum = sum(score * weight for score, weight in zip(normalized_scores, weights))
    if weighted_sum >= 0.1:
        return f"The Translation is Good"
    elif 0.05 <= weighted_sum < 0.1:
        return f"The Translation is Well"
    elif 0.02 <= weighted_sum < 0.05:
        return f"The Translation is Not Bad, Can Be Improved"
    else:
        return "Poor Translation"

# Translation Quality Checker
def translation_quality_checker(source_text, human_translation, source_lan, target_lan, task_type='translation'):
    source_lang = language_codes.get(source_lan)
    target_lang = language_codes.get(target_lan)
    model_forward, tokenizer_forward = load_translation_model(source_lang, target_lang)
    model_backward, tokenizer_backward = load_translation_model(target_lang, source_lang)

    if model_forward is None or model_backward is None:
        return "Error: Translation model for the selected language pair is unavailable."

    # Forward and Back Translation
    A_prime = translate_text(source_text, model_forward, tokenizer_forward)
    B_prime = translate_text(A_prime, model_backward, tokenizer_backward)

    # Calculate Scores
    bleu_score = max(calculate_bleu(source_text, B_prime), calculate_bleu(B_prime, human_translation))
    rouge_score = max(calculate_rouge(source_text, B_prime)["rougeL"].fmeasure, calculate_rouge(B_prime, human_translation)["rougeL"].fmeasure)
    meteor_score = max(calculate_meteor(source_text, B_prime), calculate_meteor(B_prime, human_translation))
    levenshtein_score = min(calculate_levenshtein_distance(source_text, B_prime), calculate_levenshtein_distance(B_prime, human_translation)) / max(len(source_text), len(human_translation))
    cosine_score = max(calculate_cosine_similarity(source_text, B_prime), calculate_cosine_similarity(B_prime, human_translation))
    sentence_embedding_score = max(calculate_sentence_embedding_similarity(source_text, B_prime), calculate_sentence_embedding_similarity(B_prime, human_translation))

    # Collect Scores
    individual_scores = [bleu_score, rouge_score, meteor_score, levenshtein_score, cosine_score, sentence_embedding_score]
    weights = adjust_weights_for_task(task_type)
    aggregate = aggregate_score(individual_scores, weights)
    individual_score = {"BLEU Score": bleu_score, "ROUGE Score": rouge_score, "METEOR Score": meteor_score, "Levenshtein Score": levenshtein_score, "Cosine Similarity": cosine_score, "Sentence Embedding Similarity": sentence_embedding_score}

    return A_prime, B_prime, individual_score, aggregate

# Gradio Interface
if __name__ == "__main__":
  interface = gr.Interface(
      fn=translation_quality_checker,
      inputs=[
          gr.Textbox(lines=2, placeholder="Enter the source text...", label="Source Text"),
          gr.Textbox(lines=2, placeholder="Enter the human translation...", label="Human Translation"),
          gr.Dropdown(choices=list(language_codes.keys()), label="Source Language", value="English"),
          gr.Dropdown(choices=list(language_codes.keys()), label="Target Language", value="Hindi"),
          gr.Radio(choices=['translation', 'summarization'], label="Task Type", value='translation')
      ],
      outputs=[
          gr.Textbox(label="Generated Translation (A_prime)"),
          gr.Textbox(label="Back Translation (B_prime)"),
          gr.JSON(label="Individual Scores"),
          gr.Textbox(label="Aggregate Score")
      ],
      title="Translation Quality Checker",
      description="Check the quality of machine translation by comparing source text with human translation using metrics like BLEU, ROUGE, METEOR, and more."
  )
  interface.launch()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fe7bce910a4f078671.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
